In [1]:
from lime.lime_text import LimeTextExplainer
from transformers import AutoTokenizer, AutoModelForSequenceClassification

import torch
import torch.nn.functional as F
import scipy as sp
import numpy as np

C:\Users\Tazeek\anaconda3\envs\action-lab_projects\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Note: This is for non-deep learning approaches

# Prepare everything before interpretability

# Load the dataset

# Transform the features (before it enters the model)

# Create the model and the pipeline

# LIME

This section is for intrepreting models, using LIME

In [2]:
# For loading
model_name = "nateraw/bert-base-uncased-emotion"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, do_lower_case=True)
model = AutoModelForSequenceClassification.from_pretrained(model_name, output_hidden_states=True)
labels = sorted(model.config.label2id, key=model.config.label2id.get)

class_names = labels

In [3]:
# Create the function to fetch the probability scores
def get_proba_scores(texts):

    # Tokenize
    # NOTE: This is for just one input.
    # Let's adapt for future ones
    tokenized = tokenizer(texts, padding=True, 
                return_tensors="pt",
                truncation=True)

    # Create attention mask
    #attention_mask = (tokenized != 0).type(torch.int64)

    # Get the output logits and convert to scores
    outputs = model(**tokenized)
    probability = F.softmax(outputs.logits, dim=1).detach().numpy()
    return probability

sample = 'I really did not like what I saw and I hated every moment of it.'
probability_scores = get_proba_scores(sample)
print(probability_scores)

[[0.13500156 0.0016683  0.00460634 0.8549804  0.00191366 0.0018297 ]]


In [ ]:
# Create the explainer object
explainer = LimeTextExplainer(class_names=class_names)

# Explain the instance passed in
explain_bert_output = explainer.explain_instance(sample, get_proba_scores, num_features = 5)

# Show the output in notebook
explain_bert_output.show_in_notebook(text=sample)

# SHAP

This section is for interpreting models using SHAP